In [28]:
import sys, os
import pandas as pd
sys.path.append('../../dependencies/')
import flopy as fp
import numpy as np 

In [15]:
# s = MFSimulation.load('my simulation', 'simulation.nam')  
sim = fp.mf6.MFSimulation.load(sim_ws = '../models/sgn_model_1layer_Hydraulic_Barrier', load_only=['wel'])

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    skipping package ic...
    skipping package npf...
    skipping package rch...
    skipping package riv...
    skipping package drn...
    loading package wel...
    skipping package ghb...
    skipping package obs...
    skipping package oc...
    skipping package sto...
  loading model gwt6...
    loading package disv...
    skipping package ic...
    skipping package adv...
    skipping package dsp...
    skipping package mst...
    skipping package ssm...
    skipping package oc...
    skipping package cnc...
    skipping package obs...
    skipping package gwf6-gwt6...
    skipping package ims6...
    skipping package ims6...


In [4]:
gwf = sim.get_model()
gwf.wel.stress_period_data.data.keys()

In [6]:
gwf.wel.stress_period_data.data[0]

rec.array([((0, 6230),  0.    , 'well-p1_ln'),
           ((0, 6229),  0.    , 'well-p2_ln'),
           ((0, 6236),  0.    , 'well-p3_ln'),
           ((0, 6233),  0.    , 'well-p4_ln'),
           ((0, 6665),  0.    , 'well-p5_ln'),
           ((0, 7115),  0.    , 'well-p6_ln'),
           ((0, 7575),  0.    , 'well-p7_ln'),
           ((0, 7561),  0.    , 'well-p8_ln'),
           ((0, 7101),  0.    , 'well-p9_ln'),
           ((0, 7098),  0.    , 'well-p10_ln'),
           ((0, 7081),  0.    , 'well-p11_ln'),
           ((0, 11830),  0.    , 'well-r1_ln'),
           ((0, 11843),  0.    , 'well-r2_ln'),
           ((0, 12397),  0.    , 'well-r3_ln'),
           ((0, 12409),  0.    , 'well-r4_ln'),
           ((0, 12422),  0.    , 'well-r5_ln'),
           ((0, 12969),  0.    , 'well-r6_ln'),
           ((0, 12976),  0.    , 'well-r7_ln'),
           ((0, 12989),  0.    , 'well-r8_ln'),
           ((0, 13515),  0.    , 'well-r9_ln'),
           ((0, 13522),  0.    , 'well-r10_ln'),


In [25]:
spd = pd.DataFrame.from_records(gwf.wel.stress_period_data.data[0])      
spd.to_csv("spd___________________.csv")
spd.tail()                                               # ____________ numpy Recarray to pandas DataFrame ____________

,cellid,q,boundname
139,"(0, 8671)",-0.0095,well-19/18
140,"(0, 9255)",-0.0095,well-19/19
141,"(0, 8689)",-0.0095,well-19/20
142,"(0, 7191)",-0.0095,well-19/21
143,"(0, 11259)",-0.0300,hydraulic_barrier


In [22]:
gwf.modelgrid.top_botm

array([[113.17749023, 112.95309448, 112.82367706, ...,  99.97387695,
         99.11410522,  98.89460754],
       [ 62.76613235,  62.47645187,  62.20778275, ...,  52.99921036,
         52.95099258,  52.88257599]])

In [33]:
p = gwf.modelgrid.top_botm
p2 = np.array(list(p), dtype='uint8')  # 'float32'
p2

array([[113, 112, 112, ...,  99,  99,  98],
       [ 62,  62,  62, ...,  52,  52,  52]], dtype=uint8)

# **[Link](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions)**

In [23]:
[i**3 for i in range(5)]                                          # Wells () | GRID 

[0, 1, 8, 27, 64]

In [ ]:
spd['layer'] = [i[0] for i in spd.cellid]                         # Wells ()
spd['cell'] = [i[1] for i in spd.cellid]
spd['laytop'] = gwf.modelgrid.top_botm[spd.layer,spd.cell]
spd['laybot'] = gwf.modelgrid.top_botm[spd.layer+1,spd.cell]
spd['laymidpt'] = spd.laytop-(spd.laytop - spd.laybot)/2
# spd

In [36]:
spd2 = spd
spd2.to_csv("spd_2__________________.csv")
# spd2.tail() 

### **Boundname ___ Rootname ___ New boundname**

In [39]:
spd.iloc[0].boundname

'well-p1_ln'

In [45]:
spd['rootname'] = [i.replace('\x00','').replace("well-","") for i in spd.boundname]
spd.iloc[0].rootname            # spd.dtypes

'p1_ln'

In [43]:
spd3 = spd2                                     # _______ Beta
spd3.to_csv("spd_3__________________.csv")
# spd3.tail() 

### finally, let's add the layer number to each name in the `boundname` column

In [46]:
spd.boundname = [f'{i.rstrip()}__{lay}' for i,lay in zip(spd.rootname,spd.layer)]            

In [48]:
# spd
spd4 = spd3                                     # _______ Beta
spd4.to_csv("spd_4__________________.csv")
# spd4.tail() 

### ...and save this out to a CSV file for use with `modflow-setup`

In [50]:
spd.to_csv('../data/sgn/wells_with_elev.dat')

In [51]:
spd.to_csv("spd_00__________________.csv")